In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import requests

In [2]:
# Remove comment at the beginning of the hml and also put the <html> tag in lowercase
def clean_page_to_parse(page_string):
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub(r'<HTML>',r'<html>',str(page_string))
    page_string = re.sub(r'</HTML>',r'</html>',str(page_string))

    return page_string

In [3]:
#Extract the <p> tag from a specific html link 
def extract_p_tags(html_link):
    page = urllib.request.urlopen(html_link).read()
    page = clean_page_to_parse(page)
    soup = BeautifulSoup(page,'html.parser')
    list_tp = soup.find_all('p')
    return list(set(list_tp))

In [4]:
#Extract <p> tags from link inside "html_link"  (Issue# <45)
def extract_p_tags_45(html_link):
    page_link = urllib.request.urlopen(html_link).read()
    soup_link = BeautifulSoup(page_link)
    paragraphes = soup_link.findAll('a',href = re.compile('\d+'))
    list_tp = []

    for pa in paragraphes:
        # doesn't use the link of the main page (all issues)
        if(pa['href'] != '1200000e.html'):
            html_link = 'https://enb.iisd.org/vol12/'+pa['href']
            list_tp += extract_p_tags(html_link)

    return list_tp


In [5]:
#Extract from "csv_file" all the html link to be able to extract all the <p> tags
def extract_from_csv_p_tags(csv_file):
    f = open(csv_file)
    csv_f = csv.reader(f)
    k=0
    list_pt = []
    for row in list(csv_f)[1:]:

        #Extract for 0 < Issue# < 45 
        if(int(row[4]) < 45 ):
            list_pt += extract_p_tags_45(row[6])
    
        #Extract for 66 < Issue# < 775
        else :
            request = requests.get(row[6])
            if(request.status_code == 200):
                list_pt += extract_p_tags(row[6])

    return list_pt

In [6]:
#Clean the sentence by removing special char
def clean_tp(sentence):
    s = re.sub("\r\n\s\s+"," ",sentence)
    s = re.sub("\r\n","",s)
    s = re.sub("\s\s+"," ",s)
    s = re.sub("\r\n","",s)
    return s

In [7]:
#list_meetings = extract_from_csv_p_tags('list_meetings.csv')
#list_paragraphes = [clean_tp(str(line)) for line in list_meetings ]

# generate list_p_tags.txt file
outF = open("list_paragraphes.txt", "w")
for line in list_paragraphes:
    # write line to output file
    outF.write(str(line))
    outF.write("\n")
outF.close()